# Quickstart 

Data: 1782 parallel and related timeseries 
   - Sales data for 33 product categories in 54 stores.


Use **Darts** for Time Series Forecasting

# 1.Libraries

I use the [**Darts library**](https://unit8co.github.io/darts/index.html) for all my modeling here. It is an excellent and intuitive option for forecasting in Python, especially with regards to NN models. The developing team is very helpful and answering questions in their public communication channels, so I can highly recommend the library for timeseries forecasting!

In [3]:
# Install libraries

#!pip install torch
# https://download.pytorch.org/whl/cu111/torch_stable.html

# !pip install pyyaml -U
# !pip install darts -U
# import darts
# print(darts.__version__)

# !pip install optuna -U
# %pip install --upgrade numpy



In [4]:
import numpy as np
import time
from pathlib import Path

from darts import TimeSeries
from darts.utils.timeseries_generation import gaussian_timeseries, linear_timeseries, sine_timeseries
from darts.models import LightGBMModel, CatBoostModel, Prophet, RNNModel, TFTModel, NaiveSeasonal, ExponentialSmoothing, NHiTSModel
from darts.metrics import mape, smape, rmse, rmsle
from darts.dataprocessing import Pipeline
from darts.dataprocessing.transformers import Scaler, StaticCovariatesTransformer, MissingValuesFiller, InvertibleMapper
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.statistics import check_seasonality, plot_acf, plot_residuals_analysis, plot_hist
from darts.utils.likelihood_models import QuantileRegression
from darts.utils.missing_values import fill_missing_values
# from darts.models import MovingAverage

import optuna
from optuna.integration import PyTorchLightningPruningCallback
from optuna.visualization import (
    plot_optimization_history,
    plot_contour,
    plot_param_importances,
)

from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from tqdm import tqdm

import sklearn
from sklearn import preprocessing

import pandas as pd
import torch
import matplotlib.pyplot as plt
import gc

%matplotlib inline
torch.manual_seed(1); np.random.seed(1)  # for reproducibility

/opt/anaconda3/envs/torch-gpu/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
The StatsForecast module could not be imported. To enable support for the StatsForecastAutoARIMA, StatsForecastAutoETS and Croston models, please consider installing it.


# 2.Data

# 2.1. Pre-Processing

After loading the data, I create **Darts-specific TimeSeries objects**. For the sales data, I generate so-called static covariates for each series (store number, product family, city, region, type and cluster). Those might be used by some models in Darts later on. I also create a set of time-based covariates like weekday, month and year. All covariate series get stacked together.

Furthermore, I scale all series between 0 and 1, and then apply a logarithmic transformation to them. I took this idea from [this helpful notebook](https://www.kaggle.com/code/carlmcbrideellis/store-sales-using-the-average-of-the-last-16-days). 

In [5]:
%%script echo skipping

# Kaggle data folder

# Load all Datasets
df_train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')
df_test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv')
df_holidays_events = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
df_oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv')
df_stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
df_transactions = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')
df_sample_submission = pd.read_csv('../input/store-sales-time-series-forecasting/sample_submission.csv')


skipping


In [9]:
# Local data folder

# local data path using pathlib
basepath = Path.cwd()
datapath = basepath.parent / 'data'

# raw data path
raw_datapath = datapath / 'raw'

# clearn data path
clean_datapath = datapath / 'clean'

print(raw_datapath)
print(clean_datapath)

# Load all Datasets
df_train = pd.read_csv(raw_datapath/'train.csv')
df_test = pd.read_csv(raw_datapath/'test.csv')
df_holidays_events = pd.read_csv(raw_datapath/'holidays_events.csv')
df_oil = pd.read_csv(raw_datapath/'oil.csv')
df_stores = pd.read_csv(raw_datapath/'stores.csv')
df_transactions = pd.read_csv(raw_datapath/'transactions.csv')
df_sample_submission = pd.read_csv(raw_datapath/'sample_submission.csv')


/Users/mbp14/Desktop/GoogleDrive/Kaggle/Store_sales_favorita/data/raw
/Users/mbp14/Desktop/GoogleDrive/Kaggle/Store_sales_favorita/data/clean


In [12]:
# Sales Data (Target)

family_list = df_train['family'].unique()
family_list

store_list = df_stores['store_nbr'].unique()
store_list


train_merged = pd.merge(df_train, df_stores, on ='store_nbr')
train_merged = train_merged.sort_values(["store_nbr","family","date"])
train_merged = train_merged.astype({"store_nbr":'str', "family":'str', "city":'str',
                          "state":'str', "type":'str', "cluster":'str'})


df_test_sorted = df_test.sort_values(by=['store_nbr','family'])


In [13]:
## Clustered by Product Family: Create TimeSeries objects and arrange in a Dictionary

family_TS_dict = {}

for family in family_list:
  df_family = train_merged.loc[train_merged['family'] == family]

  list_of_TS_family = TimeSeries.from_group_dataframe(
                                df_family,
                                time_col="date",
                                group_cols=["store_nbr","family"],  # individual time series are extracted by grouping `df` by `group_cols`
                                static_cols=["city","state","type","cluster"], # also extract these additional columns as static covariates
                                value_cols="sales", # target variable
                                fill_missing_dates=True,
                                freq='D')
  for ts in list_of_TS_family:
            ts = ts.astype(np.float32)

  list_of_TS_family = sorted(list_of_TS_family, key=lambda ts: int(ts.static_covariates_values()[0,0]))
  family_TS_dict[family] = list_of_TS_family


# Transform the Sales Data
family_pipeline_dict = {}
family_TS_transformed_dict = {}

for key in family_TS_dict:
  train_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
  static_cov_transformer = StaticCovariatesTransformer(verbose=False, transformer_cat = sklearn.preprocessing.OneHotEncoder(), name="Encoder") #OneHotEncoder would be better but takes longer
  log_transformer = InvertibleMapper(np.log1p, np.expm1, verbose=False, n_jobs=-1, name="Log-Transform")   
  train_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

  train_pipeline = Pipeline([train_filler,
                             static_cov_transformer,
                             log_transformer,
                             train_scaler])
     
  training_transformed = train_pipeline.fit_transform(family_TS_dict[key])
  family_pipeline_dict[key] = train_pipeline
  family_TS_transformed_dict[key] = training_transformed

# Create TimeSeries objects (Darts) 1782

list_of_TS = TimeSeries.from_group_dataframe(
            train_merged,
            time_col="date",
            group_cols=["store_nbr","family"],  # individual time series are extracted by grouping `df` by `group_cols`
            static_cols=["city","state","type","cluster"], # also extract these additional columns as static covariates
            value_cols="sales", # target variable
            fill_missing_dates=True,
            freq='D')

for ts in list_of_TS:
            ts = ts.astype(np.float32)

list_of_TS = sorted(list_of_TS, key=lambda ts: int(ts.static_covariates_values()[0,0]))

# Transform the Sales Data

train_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
static_cov_transformer = StaticCovariatesTransformer(verbose=False, transformer_cat = sklearn.preprocessing.OneHotEncoder(), name="Encoder") #OneHotEncoder would be better but takes longer
log_transformer = InvertibleMapper(np.log1p, np.expm1, verbose=False, n_jobs=-1, name="Log-Transform")   
train_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

train_pipeline = Pipeline([train_filler,
                             static_cov_transformer,
                             log_transformer,
                             train_scaler])
     
training_transformed = train_pipeline.fit_transform(list_of_TS)

#train_merged.head()


/opt/anaconda3/envs/torch-gpu/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/anaconda3/envs/torch-gpu/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/anaconda3/envs/torch-gpu/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/anaconda3/envs/torch-gpu/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  

In [ ]:

# Create 7-day and 28-day moving average of sales

sales_moving_average_7 = MovingAverage(window=7)
sales_moving_average_28 = MovingAverage(window=28)

sales_moving_averages_dict = {}

for key in family_TS_transformed_dict:
  sales_mas_family = []
  
  for ts in family_TS_transformed_dict[key]:
    ma_7 = sales_moving_average_7.filter(ts)
    ma_7 = TimeSeries.from_series(ma_7.pd_series())  
    ma_7 = ma_7.astype(np.float32)
    ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="sales_ma_7")
    ma_28 = sales_moving_average_28.filter(ts)
    ma_28 = TimeSeries.from_series(ma_28.pd_series())  
    ma_28 = ma_28.astype(np.float32)
    ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="sales_ma_28")
    mas = ma_7.stack(ma_28)
    sales_mas_family.append(mas)
  
  sales_moving_averages_dict[key] = sales_mas_family  
    
# General Covariates (Time-Based and Oil)

full_time_period = pd.date_range(start='2013-01-01', end='2017-08-31', freq='D')

# Time-Based Covariates

year = datetime_attribute_timeseries(time_index = full_time_period, attribute="year")
month = datetime_attribute_timeseries(time_index = full_time_period, attribute="month")
day = datetime_attribute_timeseries(time_index = full_time_period, attribute="day")
dayofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute="dayofyear")
weekday = datetime_attribute_timeseries(time_index = full_time_period, attribute="dayofweek")
weekofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute="weekofyear")
timesteps = TimeSeries.from_times_and_values(times=full_time_period,
                                             values=np.arange(len(full_time_period)),
                                             columns=["linear_increase"])

time_cov = year.stack(month).stack(day).stack(dayofyear).stack(weekday).stack(weekofyear).stack(timesteps)
time_cov = time_cov.astype(np.float32)

# Transform
time_cov_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")
time_cov_train, time_cov_val = time_cov.split_before(pd.Timestamp('20170816'))
time_cov_scaler.fit(time_cov_train)
time_cov_transformed = time_cov_scaler.transform(time_cov)

#time_cov_transformed[-50:].plot()

# Oil Price

oil = TimeSeries.from_dataframe(df_oil, 
                                time_col = 'date', 
                                value_cols = ['dcoilwtico'],
                                freq = 'D')

oil = oil.astype(np.float32)

# Transform
oil_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
oil_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")
oil_pipeline = Pipeline([oil_filler, oil_scaler])
oil_transformed = oil_pipeline.fit_transform(oil)

# Moving Averages for Oil Price
oil_moving_average_7 = MovingAverage(window=7)
oil_moving_average_28 = MovingAverage(window=28)

oil_moving_averages = []

ma_7 = oil_moving_average_7.filter(oil_transformed).astype(np.float32)
ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="oil_ma_7")
ma_28 = oil_moving_average_28.filter(oil_transformed).astype(np.float32)
ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="oil_ma_28")
oil_moving_averages = ma_7.stack(ma_28)

# Stack General Covariates Together

general_covariates = time_cov_transformed.stack(oil_transformed).stack(oil_moving_averages)

# Store-Specific Covariates (Transactions and Holidays)

# Transactions
df_transactions.sort_values(["store_nbr","date"], inplace=True)

TS_transactions_list = TimeSeries.from_group_dataframe(
                                df_transactions,
                                time_col="date",
                                group_cols=["store_nbr"],  # individual time series are extracted by grouping `df` by `group_cols`
                                value_cols="transactions",
                                fill_missing_dates=True,
                                freq='D')

transactions_list = []

for ts in TS_transactions_list:
            series = TimeSeries.from_series(ts.pd_series())   # necessary workaround to remove static covariates (so I can stack covariates later on)
            series = series.astype(np.float32)
            transactions_list.append(series)

transactions_list[24] = transactions_list[24].slice(start_ts=pd.Timestamp('20130102'), end_ts=pd.Timestamp('20170815'))

from datetime import datetime, timedelta

transactions_list_full = []

for ts in transactions_list:
  if ts.start_time() > pd.Timestamp('20130101'):
    end_time = (ts.start_time() - timedelta(days=1))
    delta = end_time - pd.Timestamp('20130101')
    zero_series = TimeSeries.from_times_and_values(
                              times=pd.date_range(start=pd.Timestamp('20130101'), 
                              end=end_time, freq="D"),
                              values=np.zeros(delta.days+1))
    ts = zero_series.append(ts)
    transactions_list_full.append(ts)

transactions_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
transactions_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

transactions_pipeline = Pipeline([transactions_filler, transactions_scaler])
transactions_transformed = transactions_pipeline.fit_transform(transactions_list_full)

# Moving Averages for Transactions
trans_moving_average_7 = MovingAverage(window=7)
trans_moving_average_28 = MovingAverage(window=28)

transactions_covs = []

for ts in transactions_transformed:
  ma_7 = trans_moving_average_7.filter(ts).astype(np.float32)
  ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="transactions_ma_7")
  ma_28 = trans_moving_average_28.filter(ts).astype(np.float32)
  ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="transactions_ma_28")
  trans_and_mas = ts.with_columns_renamed(col_names=ts.components, col_names_new="transactions").stack(ma_7).stack(ma_28)
  transactions_covs.append(trans_and_mas)




In [ ]:

# Re-Defining Categories of Holidays in a Meaningful Way

df_holidays_events['type'] = np.where(df_holidays_events['transferred'] == True,'Transferred', 
                                      df_holidays_events['type'])

df_holidays_events['type'] = np.where(df_holidays_events['type'] == 'Transfer','Holiday', 
                                      df_holidays_events['type'])

df_holidays_events['type'] = np.where(df_holidays_events['type'] == 'Additional','Holiday', 
                                      df_holidays_events['type'])

df_holidays_events['type'] = np.where(df_holidays_events['type'] == 'Bridge','Holiday', 
                                      df_holidays_events['type'])


# Assign Holidays to all TimeSeries and Save in Dictionary

def holiday_list(df_stores):

    listofseries = []
    
    for i in range(0,len(df_stores)):
            
            df_holiday_dummies = pd.DataFrame(columns=['date'])
            df_holiday_dummies["date"] = df_holidays_events["date"]
            
            df_holiday_dummies["national_holiday"] = np.where(((df_holidays_events["type"] == "Holiday") & (df_holidays_events["locale"] == "National")), 1, 0)

            df_holiday_dummies["earthquake_relief"] = np.where(df_holidays_events['description'].str.contains('Terremoto Manabi'), 1, 0)

            df_holiday_dummies["christmas"] = np.where(df_holidays_events['description'].str.contains('Navidad'), 1, 0)

            df_holiday_dummies["football_event"] = np.where(df_holidays_events['description'].str.contains('futbol'), 1, 0)

            df_holiday_dummies["national_event"] = np.where(((df_holidays_events["type"] == "Event") & (df_holidays_events["locale"] == "National") & (~df_holidays_events['description'].str.contains('Terremoto Manabi')) & (~df_holidays_events['description'].str.contains('futbol'))), 1, 0)

            df_holiday_dummies["work_day"] = np.where((df_holidays_events["type"] == "Work Day"), 1, 0)

            df_holiday_dummies["local_holiday"] = np.where(((df_holidays_events["type"] == "Holiday") & ((df_holidays_events["locale_name"] == df_stores['state'][i]) | (df_holidays_events["locale_name"] == df_stores['city'][i]))), 1, 0)
                     
            listofseries.append(df_holiday_dummies)

    return listofseries

def remove_0_and_duplicates(holiday_list):

    listofseries = []
    
    for i in range(0,len(holiday_list)):
            
            df_holiday_per_store = list_of_holidays_per_store[i].set_index('date')

            df_holiday_per_store = df_holiday_per_store.loc[~(df_holiday_per_store==0).all(axis=1)]
            
            df_holiday_per_store = df_holiday_per_store.groupby('date').agg({'national_holiday':'max', 'earthquake_relief':'max', 
                                   'christmas':'max', 'football_event':'max', 
                                   'national_event':'max', 'work_day':'max', 
                                   'local_holiday':'max'}).reset_index()

            listofseries.append(df_holiday_per_store)

    return listofseries

def holiday_TS_list_54(holiday_list):

    listofseries = []
    
    for i in range(0,54):
            
            holidays_TS = TimeSeries.from_dataframe(list_of_holidays_per_store[i], 
                                        time_col = 'date',
                                        fill_missing_dates=True,
                                        fillna_value=0,
                                        freq='D')
            
            holidays_TS = holidays_TS.slice(pd.Timestamp('20130101'),pd.Timestamp('20170831'))
            holidays_TS = holidays_TS.astype(np.float32)
            listofseries.append(holidays_TS)

    return listofseries


list_of_holidays_per_store = holiday_list(df_stores)
list_of_holidays_per_store = remove_0_and_duplicates(list_of_holidays_per_store)   
list_of_holidays_store = holiday_TS_list_54(list_of_holidays_per_store)

holidays_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
holidays_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

holidays_pipeline = Pipeline([holidays_filler, holidays_scaler])
holidays_transformed = holidays_pipeline.fit_transform(list_of_holidays_store)



In [ ]:

# Stack Together Store-Specific Covariates with General Covariates

store_covariates_future = []

for store in range(0,len(store_list)):
  stacked_covariates = holidays_transformed[store].stack(general_covariates)  
  store_covariates_future.append(stacked_covariates)

store_covariates_past = []
holidays_transformed_sliced = holidays_transformed # for slicing past covariates

for store in range(0,len(store_list)):
  holidays_transformed_sliced[store] = holidays_transformed[store].slice_intersect(transactions_covs[store])
  general_covariates_sliced = general_covariates.slice_intersect(transactions_covs[store])
  stacked_covariates = transactions_covs[store].stack(holidays_transformed_sliced[store]).stack(general_covariates_sliced)  
  store_covariates_past.append(stacked_covariates)
    
# Store/Family-Varying Covariates (Promotion)

df_promotion = pd.concat([df_train, df_test], axis=0)
df_promotion = df_promotion.sort_values(["store_nbr","family","date"])
df_promotion.tail()

family_promotion_dict = {}

for family in family_list:
  df_family = df_promotion.loc[df_promotion['family'] == family]

  list_of_TS_promo = TimeSeries.from_group_dataframe(
                                df_family,
                                time_col="date",
                                group_cols=["store_nbr","family"],  # individual time series are extracted by grouping `df` by `group_cols`
                                value_cols="onpromotion", # covariate of interest
                                fill_missing_dates=True,
                                freq='D')
  
  for ts in list_of_TS_promo:
            ts = ts.astype(np.float32)

  family_promotion_dict[family] = list_of_TS_promo

promotion_transformed_dict = {}

for key in tqdm(family_promotion_dict):
  promo_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
  promo_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

  promo_pipeline = Pipeline([promo_filler,
                             promo_scaler])
  
  promotion_transformed = promo_pipeline.fit_transform(family_promotion_dict[key])

  # Moving Averages for Promotion Family Dictionaries
  promo_moving_average_7 = MovingAverage(window=7)
  promo_moving_average_28 = MovingAverage(window=28)

  promotion_covs = []

  for ts in promotion_transformed:
    ma_7 = promo_moving_average_7.filter(ts)
    ma_7 = TimeSeries.from_series(ma_7.pd_series())  
    ma_7 = ma_7.astype(np.float32)
    ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="promotion_ma_7")
    ma_28 = promo_moving_average_28.filter(ts)
    ma_28 = TimeSeries.from_series(ma_28.pd_series())  
    ma_28 = ma_28.astype(np.float32)
    ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="promotion_ma_28")
    promo_and_mas = ts.stack(ma_7).stack(ma_28)
    promotion_covs.append(promo_and_mas)

  promotion_transformed_dict[key] = promotion_covs


In [ ]:

# 2.5. Assemble All Covariates in Dictionaries

past_covariates_dict = {}

for key in tqdm(promotion_transformed_dict):

  promotion_family = promotion_transformed_dict[key]
  sales_mas = sales_moving_averages_dict[key]
  covariates_past = [promotion_family[i].slice_intersect(store_covariates_past[i]).stack(store_covariates_past[i].stack(sales_mas[i])) for i in range(0,len(promotion_family))]

  past_covariates_dict[key] = covariates_past

future_covariates_dict = {}

for key in tqdm(promotion_transformed_dict):

  promotion_family = promotion_transformed_dict[key]
  covariates_future = [promotion_family[i].stack(store_covariates_future[i]) for i in range(0,len(promotion_family))]

  future_covariates_dict[key] = covariates_future

only_past_covariates_dict = {}

for key in tqdm(sales_moving_averages_dict):
  sales_moving_averages = sales_moving_averages_dict[key]
  only_past_covariates = [sales_moving_averages[i].stack(transactions_covs[i]) for i in range(0,len(sales_moving_averages))]

  only_past_covariates_dict[key] = only_past_covariates

# Delete Original Dataframes to Save Memory

del(df_train)
del(df_test)
del(df_stores)
del(df_holidays_events)
del(df_oil)
del(df_transactions)
gc.collect()



In [ ]:
%%script echo skipping

# Sales Data (Target)

family_list = df_train['family'].unique()
family_list
store_list = df_stores['store_nbr'].unique()
store_list

train_merged = pd.merge(df_train, df_stores, on ='store_nbr')
train_merged = train_merged.sort_values(["store_nbr","family","date"])
train_merged = train_merged.astype({"store_nbr":'str', "family":'str', "city":'str',
                          "state":'str', "type":'str', "cluster":'str'})

df_test_dropped = df_test.drop(['onpromotion'], axis=1)
df_test_sorted = df_test_dropped.sort_values(by=['store_nbr','family'])

# Create TimeSeries objects (Darts) and arrange in a Dictionary clustered by Product Family

family_TS_dict = {}

for family in family_list:
  df_family = train_merged.loc[train_merged['family'] == family]

  list_of_TS_family = TimeSeries.from_group_dataframe(
                                df_family,
                                time_col="date",
                                group_cols=["store_nbr","family"],  # individual time series are extracted by grouping `df` by `group_cols`
                                static_cols=["city","state","type","cluster"], # also extract these additional columns as static covariates
                                value_cols="sales", # target variable
                                fill_missing_dates=True,
                                freq='D')
  for ts in list_of_TS_family:
            ts = ts.astype(np.float32)

  list_of_TS_family = sorted(list_of_TS_family, key=lambda ts: int(ts.static_covariates_values()[0,0]))
  family_TS_dict[family] = list_of_TS_family

# Transform the Sales Data

family_pipeline_dict = {}
family_TS_transformed_dict = {}

for key in family_TS_dict:
  train_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
  static_cov_transformer = StaticCovariatesTransformer(verbose=False, transformer_cat = sklearn.preprocessing.OneHotEncoder(), name="Encoder") #OneHotEncoder would be better but takes longer
  log_transformer = InvertibleMapper(np.log1p, np.expm1, verbose=False, n_jobs=-1, name="Log-Transform")   
  train_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

  train_pipeline = Pipeline([train_filler,
                             static_cov_transformer,
                             log_transformer,
                             train_scaler])
     
  training_transformed = train_pipeline.fit_transform(family_TS_dict[key])
  family_pipeline_dict[key] = train_pipeline
  family_TS_transformed_dict[key] = training_transformed

# Create TimeSeries objects (Darts) 1782

list_of_TS = TimeSeries.from_group_dataframe(
                                train_merged,
                                time_col="date",
                                group_cols=["store_nbr","family"],  # individual time series are extracted by grouping `df` by `group_cols`
                                static_cols=["city","state","type","cluster"], # also extract these additional columns as static covariates
                                value_cols="sales", # target variable
                                fill_missing_dates=True,
                                freq='D')
for ts in list_of_TS:
            ts = ts.astype(np.float32)

list_of_TS = sorted(list_of_TS, key=lambda ts: int(ts.static_covariates_values()[0,0]))

# Transform the Sales Data

train_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
static_cov_transformer = StaticCovariatesTransformer(verbose=False, transformer_cat = sklearn.preprocessing.OneHotEncoder(), name="Encoder") #OneHotEncoder would be better but takes longer
log_transformer = InvertibleMapper(np.log1p, np.expm1, verbose=False, n_jobs=-1, name="Log-Transform")   
train_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

train_pipeline = Pipeline([train_filler,
                             static_cov_transformer,
                             log_transformer,
                             train_scaler])
     
training_transformed = train_pipeline.fit_transform(list_of_TS)

#train_merged.head()

# Create 7-day and 28-day moving average of sales

sales_moving_average_7 = MovingAverage(window=7)
sales_moving_average_28 = MovingAverage(window=28)

sales_moving_averages_dict = {}

for key in family_TS_transformed_dict:
  sales_mas_family = []
  
  for ts in family_TS_transformed_dict[key]:
    ma_7 = sales_moving_average_7.filter(ts)
    ma_7 = TimeSeries.from_series(ma_7.pd_series())  
    ma_7 = ma_7.astype(np.float32)
    ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="sales_ma_7")
    ma_28 = sales_moving_average_28.filter(ts)
    ma_28 = TimeSeries.from_series(ma_28.pd_series())  
    ma_28 = ma_28.astype(np.float32)
    ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="sales_ma_28")
    mas = ma_7.stack(ma_28)
    sales_mas_family.append(mas)
  
  sales_moving_averages_dict[key] = sales_mas_family  
    
# General Covariates (Time-Based and Oil)

full_time_period = pd.date_range(start='2013-01-01', end='2017-08-31', freq='D')

# Time-Based Covariates

year = datetime_attribute_timeseries(time_index = full_time_period, attribute="year")
month = datetime_attribute_timeseries(time_index = full_time_period, attribute="month")
day = datetime_attribute_timeseries(time_index = full_time_period, attribute="day")
dayofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute="dayofyear")
weekday = datetime_attribute_timeseries(time_index = full_time_period, attribute="dayofweek")
weekofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute="weekofyear")
timesteps = TimeSeries.from_times_and_values(times=full_time_period,
                                             values=np.arange(len(full_time_period)),
                                             columns=["linear_increase"])

time_cov = year.stack(month).stack(day).stack(dayofyear).stack(weekday).stack(weekofyear).stack(timesteps)
time_cov = time_cov.astype(np.float32)

# Transform
time_cov_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")
time_cov_train, time_cov_val = time_cov.split_before(pd.Timestamp('20170816'))
time_cov_scaler.fit(time_cov_train)
time_cov_transformed = time_cov_scaler.transform(time_cov)

#time_cov_transformed[-50:].plot()

# Oil Price

oil = TimeSeries.from_dataframe(df_oil, 
                                time_col = 'date', 
                                value_cols = ['dcoilwtico'],
                                freq = 'D')

oil = oil.astype(np.float32)

# Transform
oil_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
oil_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")
oil_pipeline = Pipeline([oil_filler, oil_scaler])
oil_transformed = oil_pipeline.fit_transform(oil)

# Moving Averages for Oil Price
oil_moving_average_7 = MovingAverage(window=7)
oil_moving_average_28 = MovingAverage(window=28)

oil_moving_averages = []

ma_7 = oil_moving_average_7.filter(oil_transformed).astype(np.float32)
ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="oil_ma_7")
ma_28 = oil_moving_average_28.filter(oil_transformed).astype(np.float32)
ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="oil_ma_28")
oil_moving_averages = ma_7.stack(ma_28)

# Stack General Covariates Together

general_covariates = time_cov_transformed.stack(oil_transformed).stack(oil_moving_averages)

# Store-Specific Covariates (Transactions and Holidays)

# Transactions
df_transactions.sort_values(["store_nbr","date"], inplace=True)

TS_transactions_list = TimeSeries.from_group_dataframe(
                                df_transactions,
                                time_col="date",
                                group_cols=["store_nbr"],  # individual time series are extracted by grouping `df` by `group_cols`
                                value_cols="transactions",
                                fill_missing_dates=True,
                                freq='D')

transactions_list = []

for ts in TS_transactions_list:
            series = TimeSeries.from_series(ts.pd_series())   # necessary workaround to remove static covariates (so I can stack covariates later on)
            series = series.astype(np.float32)
            transactions_list.append(series)

transactions_list[24] = transactions_list[24].slice(start_ts=pd.Timestamp('20130102'), end_ts=pd.Timestamp('20170815'))

from datetime import datetime, timedelta

transactions_list_full = []

for ts in transactions_list:
  if ts.start_time() > pd.Timestamp('20130101'):
    end_time = (ts.start_time() - timedelta(days=1))
    delta = end_time - pd.Timestamp('20130101')
    zero_series = TimeSeries.from_times_and_values(
                              times=pd.date_range(start=pd.Timestamp('20130101'), 
                              end=end_time, freq="D"),
                              values=np.zeros(delta.days+1))
    ts = zero_series.append(ts)
    transactions_list_full.append(ts)

transactions_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
transactions_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

transactions_pipeline = Pipeline([transactions_filler, transactions_scaler])
transactions_transformed = transactions_pipeline.fit_transform(transactions_list_full)

# Moving Averages for Transactions
trans_moving_average_7 = MovingAverage(window=7)
trans_moving_average_28 = MovingAverage(window=28)

transactions_covs = []

for ts in transactions_transformed:
  ma_7 = trans_moving_average_7.filter(ts).astype(np.float32)
  ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="transactions_ma_7")
  ma_28 = trans_moving_average_28.filter(ts).astype(np.float32)
  ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="transactions_ma_28")
  trans_and_mas = ts.with_columns_renamed(col_names=ts.components, col_names_new="transactions").stack(ma_7).stack(ma_28)
  transactions_covs.append(trans_and_mas)



# Re-Defining Categories of Holidays in a Meaningful Way

df_holidays_events['type'] = np.where(df_holidays_events['transferred'] == True,'Transferred', 
                                      df_holidays_events['type'])

df_holidays_events['type'] = np.where(df_holidays_events['type'] == 'Transfer','Holiday', 
                                      df_holidays_events['type'])

df_holidays_events['type'] = np.where(df_holidays_events['type'] == 'Additional','Holiday', 
                                      df_holidays_events['type'])

df_holidays_events['type'] = np.where(df_holidays_events['type'] == 'Bridge','Holiday', 
                                      df_holidays_events['type'])


# Assign Holidays to all TimeSeries and Save in Dictionary

def holiday_list(df_stores):

    listofseries = []
    
    for i in range(0,len(df_stores)):
            
            df_holiday_dummies = pd.DataFrame(columns=['date'])
            df_holiday_dummies["date"] = df_holidays_events["date"]
            
            df_holiday_dummies["national_holiday"] = np.where(((df_holidays_events["type"] == "Holiday") & (df_holidays_events["locale"] == "National")), 1, 0)

            df_holiday_dummies["earthquake_relief"] = np.where(df_holidays_events['description'].str.contains('Terremoto Manabi'), 1, 0)

            df_holiday_dummies["christmas"] = np.where(df_holidays_events['description'].str.contains('Navidad'), 1, 0)

            df_holiday_dummies["football_event"] = np.where(df_holidays_events['description'].str.contains('futbol'), 1, 0)

            df_holiday_dummies["national_event"] = np.where(((df_holidays_events["type"] == "Event") & (df_holidays_events["locale"] == "National") & (~df_holidays_events['description'].str.contains('Terremoto Manabi')) & (~df_holidays_events['description'].str.contains('futbol'))), 1, 0)

            df_holiday_dummies["work_day"] = np.where((df_holidays_events["type"] == "Work Day"), 1, 0)

            df_holiday_dummies["local_holiday"] = np.where(((df_holidays_events["type"] == "Holiday") & ((df_holidays_events["locale_name"] == df_stores['state'][i]) | (df_holidays_events["locale_name"] == df_stores['city'][i]))), 1, 0)
                     
            listofseries.append(df_holiday_dummies)

    return listofseries

def remove_0_and_duplicates(holiday_list):

    listofseries = []
    
    for i in range(0,len(holiday_list)):
            
            df_holiday_per_store = list_of_holidays_per_store[i].set_index('date')

            df_holiday_per_store = df_holiday_per_store.loc[~(df_holiday_per_store==0).all(axis=1)]
            
            df_holiday_per_store = df_holiday_per_store.groupby('date').agg({'national_holiday':'max', 'earthquake_relief':'max', 
                                   'christmas':'max', 'football_event':'max', 
                                   'national_event':'max', 'work_day':'max', 
                                   'local_holiday':'max'}).reset_index()

            listofseries.append(df_holiday_per_store)

    return listofseries

def holiday_TS_list_54(holiday_list):

    listofseries = []
    
    for i in range(0,54):
            
            holidays_TS = TimeSeries.from_dataframe(list_of_holidays_per_store[i], 
                                        time_col = 'date',
                                        fill_missing_dates=True,
                                        fillna_value=0,
                                        freq='D')
            
            holidays_TS = holidays_TS.slice(pd.Timestamp('20130101'),pd.Timestamp('20170831'))
            holidays_TS = holidays_TS.astype(np.float32)
            listofseries.append(holidays_TS)

    return listofseries


list_of_holidays_per_store = holiday_list(df_stores)
list_of_holidays_per_store = remove_0_and_duplicates(list_of_holidays_per_store)   
list_of_holidays_store = holiday_TS_list_54(list_of_holidays_per_store)

holidays_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
holidays_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

holidays_pipeline = Pipeline([holidays_filler, holidays_scaler])
holidays_transformed = holidays_pipeline.fit_transform(list_of_holidays_store)

# Stack Together Store-Specific Covariates with General Covariates

store_covariates_future = []

for store in range(0,len(store_list)):
  stacked_covariates = holidays_transformed[store].stack(general_covariates)  
  store_covariates_future.append(stacked_covariates)

store_covariates_past = []
holidays_transformed_sliced = holidays_transformed # for slicing past covariates

for store in range(0,len(store_list)):
  holidays_transformed_sliced[store] = holidays_transformed[store].slice_intersect(transactions_covs[store])
  general_covariates_sliced = general_covariates.slice_intersect(transactions_covs[store])
  stacked_covariates = transactions_covs[store].stack(holidays_transformed_sliced[store]).stack(general_covariates_sliced)  
  store_covariates_past.append(stacked_covariates)
    
# Store/Family-Varying Covariates (Promotion)

df_promotion = pd.concat([df_train, df_test], axis=0)
df_promotion = df_promotion.sort_values(["store_nbr","family","date"])
df_promotion.tail()

family_promotion_dict = {}

for family in family_list:
  df_family = df_promotion.loc[df_promotion['family'] == family]

  list_of_TS_promo = TimeSeries.from_group_dataframe(
                                df_family,
                                time_col="date",
                                group_cols=["store_nbr","family"],  # individual time series are extracted by grouping `df` by `group_cols`
                                value_cols="onpromotion", # covariate of interest
                                fill_missing_dates=True,
                                freq='D')
  
  for ts in list_of_TS_promo:
            ts = ts.astype(np.float32)

  family_promotion_dict[family] = list_of_TS_promo

promotion_transformed_dict = {}

for key in tqdm(family_promotion_dict):
  promo_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
  promo_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

  promo_pipeline = Pipeline([promo_filler,
                             promo_scaler])
  
  promotion_transformed = promo_pipeline.fit_transform(family_promotion_dict[key])

  # Moving Averages for Promotion Family Dictionaries
  promo_moving_average_7 = MovingAverage(window=7)
  promo_moving_average_28 = MovingAverage(window=28)

  promotion_covs = []

  for ts in promotion_transformed:
    ma_7 = promo_moving_average_7.filter(ts)
    ma_7 = TimeSeries.from_series(ma_7.pd_series())  
    ma_7 = ma_7.astype(np.float32)
    ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="promotion_ma_7")
    ma_28 = promo_moving_average_28.filter(ts)
    ma_28 = TimeSeries.from_series(ma_28.pd_series())  
    ma_28 = ma_28.astype(np.float32)
    ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="promotion_ma_28")
    promo_and_mas = ts.stack(ma_7).stack(ma_28)
    promotion_covs.append(promo_and_mas)

  promotion_transformed_dict[key] = promotion_covs

# 2.5. Assemble All Covariates in Dictionaries

past_covariates_dict = {}

for key in tqdm(promotion_transformed_dict):

  promotion_family = promotion_transformed_dict[key]
  sales_mas = sales_moving_averages_dict[key]
  covariates_past = [promotion_family[i].slice_intersect(store_covariates_past[i]).stack(store_covariates_past[i].stack(sales_mas[i])) for i in range(0,len(promotion_family))]

  past_covariates_dict[key] = covariates_past

future_covariates_dict = {}

for key in tqdm(promotion_transformed_dict):

  promotion_family = promotion_transformed_dict[key]
  covariates_future = [promotion_family[i].stack(store_covariates_future[i]) for i in range(0,len(promotion_family))]

  future_covariates_dict[key] = covariates_future

only_past_covariates_dict = {}

for key in tqdm(sales_moving_averages_dict):
  sales_moving_averages = sales_moving_averages_dict[key]
  only_past_covariates = [sales_moving_averages[i].stack(transactions_covs[i]) for i in range(0,len(sales_moving_averages))]

  only_past_covariates_dict[key] = only_past_covariates

# Delete Original Dataframes to Save Memory

del(df_train)
del(df_test)
del(df_stores)
del(df_holidays_events)
del(df_oil)
del(df_transactions)
gc.collect()



<a id="2.2."></a> <br>
# 2.2. EDA

For a first impression, let's look at a few of the 1782 (store x product family) timeseries:

In [ ]:
# Some EDA

bread_series = family_TS_dict['BREAD/BAKERY'][0]
celebration_series = family_TS_dict['CELEBRATION'][11]


# Let's print two of the 1782 TimeSeries

plt.subplots(2, 2, figsize=(15, 6))
plt.subplot(1, 2, 1) # row 1, col 2 index 1
bread_series.plot(label='Sales for {}'.format(bread_series.static_covariates_values()[0,1], 
                                                bread_series.static_covariates_values()[0,0],
                                                bread_series.static_covariates_values()[0,2]))

celebration_series.plot(label='Sales for {}'.format(celebration_series.static_covariates_values()[0,1], 
                                                celebration_series.static_covariates_values()[0,0],
                                                celebration_series.static_covariates_values()[0,2]))

plt.title("Two Out Of 1782 TimeSeries")
           
plt.subplot(1, 2, 2) # index 2
bread_series[-365:].plot(label='Sales for {}'.format(bread_series.static_covariates_values()[0,1], 
                                                bread_series.static_covariates_values()[0,0],
                                                bread_series.static_covariates_values()[0,2]))

celebration_series[-365:].plot(label='Sales for {}'.format(celebration_series.static_covariates_values()[0,1], 
                                                celebration_series.static_covariates_values()[0,0],
                                                celebration_series.static_covariates_values()[0,2]))

plt.title("Only The Last 365 Days")
plt.show()

Let's also plot the ACF for both series and investigate the seasonal patterns:

In [ ]:
# Inspect Seasonality

plot_acf(fill_missing_values(bread_series), m=7, alpha=0.05)
plt.title("{}, store {} in {}".format(bread_series.static_covariates_values()[0,1], 
                                                bread_series.static_covariates_values()[0,0],
                                                bread_series.static_covariates_values()[0,2]))

plot_acf(fill_missing_values(celebration_series), alpha=0.05)
plt.title("{}, store {} in {}".format(celebration_series.static_covariates_values()[0,1], 
                                                celebration_series.static_covariates_values()[0,0],
                                                celebration_series.static_covariates_values()[0,2]));

As we can see, the BREAD/BAKERY series displays strong weekly seasonality, as we would expect. The CELEBRATION series however has a much less clear seasonal pattern. 

I encoded the static covariates and applied 0-1 Scaling + Log-Transformation to all series. Static covariates don't vary over time - examples in our dataset are the store number or region. Scaling is important for many of the deep learning models, and the logarithmic transformation of the training data will help against undershooting the actual sales with our forecasts.

In [ ]:
# Show the Differenced Series

# First Transform the Example Series
train_filler_bread = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
static_cov_transformer_bread = StaticCovariatesTransformer(verbose=False, transformer_cat = sklearn.preprocessing.OneHotEncoder(), name="Encoder") 
log_transformer_bread = InvertibleMapper(np.log1p, np.expm1, verbose=False, n_jobs=-1, name="Log-Transform")   
train_scaler_bread = Scaler(verbose=False, n_jobs=-1, name="Scaling")

train_filler_celebration = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
static_cov_transformer_celebration = StaticCovariatesTransformer(verbose=False, transformer_cat = sklearn.preprocessing.OneHotEncoder(), name="Encoder") 
log_transformer_celebration = InvertibleMapper(np.log1p, np.expm1, verbose=False, n_jobs=-1, name="Log-Transform")   
train_scaler_celebration = Scaler(verbose=False, n_jobs=-1, name="Scaling")

train_pipeline_bread = Pipeline([train_filler_bread,
                             static_cov_transformer_bread,
                             log_transformer_bread,
                             train_scaler_bread])

train_pipeline_celebration = Pipeline([train_filler_celebration,
                             static_cov_transformer_celebration,
                             log_transformer_celebration,
                             train_scaler_celebration])
     
bread_series_transformed = train_pipeline_bread.fit_transform(bread_series)
celebration_series_transformed = train_pipeline_celebration.fit_transform(celebration_series)

# Plots

plt.subplots(2, 2, figsize=(15, 6))
plt.subplot(1, 2, 1) # row 1, col 2 index 1
bread_series_transformed.plot(label='Sales for {}'.format(bread_series.static_covariates_values()[0,1], 
                                                bread_series.static_covariates_values()[0,0],
                                                bread_series.static_covariates_values()[0,2]))

plt.title("TimeSeries After Scaling and Log-Transform")
           
plt.subplot(1, 2, 2) # index 2
bread_series_transformed[-365:].plot(label='Sales for {}'.format(bread_series.static_covariates_values()[0,1], 
                                                bread_series.static_covariates_values()[0,0],
                                                bread_series.static_covariates_values()[0,2]))

plt.title("Only The Last 365 Days")
plt.show()

plt.subplots(2, 2, figsize=(15, 6))
plt.subplot(1, 2, 1) # row 1, col 2 index 1
celebration_series_transformed.plot(label='Sales for {}'.format(celebration_series.static_covariates_values()[0,1], 
                                                celebration_series.static_covariates_values()[0,0],
                                                celebration_series.static_covariates_values()[0,2]))

plt.title("TimeSeries After Scaling and Log-Transform")
           
plt.subplot(1, 2, 2) # index 2
celebration_series_transformed[-365:].plot(label='Sales for {}'.format(celebration_series.static_covariates_values()[0,1], 
                                                celebration_series.static_covariates_values()[0,0],
                                                celebration_series.static_covariates_values()[0,2]))

plt.title("Only The Last 365 Days")
plt.show()

## Covariates

Let's look at the covariates for the last 180 days of the BREAD/BAKERY series in store 1.

### Sales Moving Average Terms


In [ ]:
plt.figure(figsize=(10, 6))
family_TS_transformed_dict['BREAD/BAKERY'][0][-180:].plot()
sales_moving_averages_dict['BREAD/BAKERY'][0][-180:].plot()
plt.title("Sales 7- and 28-day Moving Averages");

### Promotion Data



In [ ]:
plt.figure(figsize=(10, 6))
promotion_transformed_dict['BREAD/BAKERY'][0][-180:].plot()
plt.title("Promotion Data and Moving Averages");

### Transactions

In [ ]:
plt.figure(figsize=(10, 6))
transactions_covs[0][-180:].plot()
plt.title("Transactions Data and Moving Averages");

### Oil Price



In [ ]:
plt.figure(figsize=(10, 6))
oil_transformed[-180:].plot()
oil_moving_averages[-180:].plot()
plt.title("Oil Price and Moving Averages");

### Time Dummies and Covariates



In [ ]:
plt.figure(figsize=(10, 6))
time_cov_transformed[-180:].plot()
plt.title("Time-Related Covariates");

### Holidays and Events



I ordered the available data on holidays in the following seven categories. I think it might work better to generalize the categories further than what I did here.

In [ ]:
#df_holidays_events['type'].value_counts().plot.bar(rot=0)
plt.figure(figsize=(10, 6))
list_of_holidays_per_store[0].loc[:, list_of_holidays_per_store[0].columns != "date"].sum().plot.bar(rot=0)
plt.title("Holidays and Events");